<a href="https://colab.research.google.com/github/federicoding/segmentation_one/blob/main/OHE_and_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Customer segmentation

Libre inspiration (donc copie) de [One Hot Encoding etc etc...](https://towardsdatascience.com/one-hot-encoding-standardization-pca-data-preparation-steps-for-segmentation-in-python-24d07671cf0b).

Histoire de faire, dans un premier temps, des trucs cleans et carrés.

Avec, bien entendu, un premier petit détour pour le storage des données.
Je vais explorer la partie GitHub, en relançant mon compte et en y stockant les fichiers en question (à priori petits de toutes façons), que j'importerai un peu comme je l'ai fait pour le projet DreamQuark.

Cool ça a marché rapidement!

In [ ]:
# Read dataset and look at top records
import pandas as pd
df = pd.read_csv('e-shop clothing 2008.csv', delimiter=";")
df.head(5)

In [ ]:
url='https://raw.githubusercontent.com/dreamquark-ai/data-science-test/master/data/requests_train.csv'
requests_train = pd.read_csv(filepath_or_buffer=url,
                             sep=',',
                             low_memory=False,
                             error_bad_lines=False)

In [ ]:
import pandas as pd

In [ ]:
url='https://raw.githubusercontent.com/federicoding/segmentation_one/main/e-shop%20clothing%202008.csv'
df = pd.read_csv(filepath_or_buffer=url,
                 sep=';')

In [ ]:
df.head(5)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


On a les données journalière pour différents types de vêtements (field name: "page2(clothing model)").
Vérifions maintenant le nombre de lignes et de colonnes et leur types (en utilisant `info`)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  int64 
 1   month                    165474 non-null  int64 
 2   day                      165474 non-null  int64 
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  int64 
 5   session ID               165474 non-null  int64 
 6   page 1 (main category)   165474 non-null  int64 
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  int64 
 9   location                 165474 non-null  int64 
 10  model photography        165474 non-null  int64 
 11  price                    165474 non-null  int64 
 12  price 2                  165474 non-null  int64 
 13  page                     165474 non-null  int64 
dtypes: int64(13), object

On a 165474 entrées et 14 champs. La grande majorité est numérique et on les voudrait en strings. On va donc les convertir en utilisant la fonction `as.type(str)`

In [ ]:
cat_vars = ['year', 'month', 'day', 'country', 'session ID', 'page 1 (main category)',
            'page 2 (clothing model)', 'colour', 'location', 'model photography',
             'price 2', 'page']
df[cat_vars] = df[cat_vars].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  object
 1   month                    165474 non-null  object
 2   day                      165474 non-null  object
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  object
 5   session ID               165474 non-null  object
 6   page 1 (main category)   165474 non-null  object
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  object
 9   location                 165474 non-null  object
 10  model photography        165474 non-null  object
 11  price                    165474 non-null  int64 
 12  price 2                  165474 non-null  object
 13  page                     165474 non-null  object
dtypes: int64(2), object(

Vérifions les propriétés des champs numériques

In [ ]:
df.describe()

,order,price
count,165474.000000,165474.000000
mean,9.817476,43.802507
std,13.478411,12.548131
min,1.000000,18.000000
25%,2.000000,33.000000
50%,6.000000,43.000000
75%,12.000000,52.000000
max,195.000000,82.000000


Comme on peut le voir, les prix des produits (field name `price` sont sur un échelle plus grande que le nombre de clics en une session (field name `order`).
Du coup on va devoir standardiser ces champs pour les ramener à la même échelle, car les modèles basés sur la distance comme les K-means sont affectés par l'échelle des champs.

#Feature Engineering

Comme mentionné précedemment, notre jeu de données est sur une échelle journalière et il nous faut agréger les données au niveau d'un produit car l'on veut réaliser une segmentation par produit.
On crée donc les **features** suivantes tout en agrégeant au niveau du produit:

1.   Le/la plus fréquent(e) couleur du produit, jour de navigation, pays, type de photo (de profil, de face), type de prix (supérieur ou inférieur à la moyenne de la catégorie), nombre de pages dans le site web et localisation de la photo du produit sur la page (en utilisant la fonction `mode`)

2.   Nombre total de IDs sessions uniques (en utilisant la fonction `nunique`)

3.   Minimum, Maximum et Mediane de la sequence de clics pendant une session et un prix de produit (en utilisant les fonctions `min`, `max` et `median`)



In [ ]:
from scipy.stats import mode

df2 = df.groupby(['country','page 1 (main category)','page 2 (clothing model)']).agg(
    
    median_no_of_clicks_per_session = ('order', 'median'),

    min_no_of_clicks_per_session = ('order', 'min'),

    max_no_of_clicks_per_session = ('order', 'max'),

    median_price = ('price', 'median'),

    min_price = ('price', 'min'),

    max_price = ('price', 'max'),

    total_number_of_sessions = ('session ID', pd.Series.nunique),

    most_frequent_day = ('day', lambda x: mode(x)[0][0]),

    most_frequent_colour = ('colour', lambda x: mode(x)[0][0]),

    most_frequent_location = ('location', lambda x: mode(x)[0][0]),

    most_frequent_photo_type = ('model photography', lambda x: mode(x)[0][0]),

    most_frequent_price_type = ('price 2', lambda x: mode(x)[0][0]),

    most_frequent_page_number = ('page', lambda x: mode(x)[0][0])
)

df2

median_no_of_clicks_per_session  ...  most_frequent_page_number
country page 1 (main category) page 2 (clothing model)                                   ...                           
1       3                      C30                                                  2.0  ...                          2
        4                      P17                                                  2.0  ...                          1
10      1                      A1                                                   2.0  ...                          1
                               A11                                                  2.0  ...                          1
                               A12                                                  3.0  ...                          1
...                                                                                 ...  ...                        ...
9       4                      P77                                                 18.0  ...                          5
                               P78                                                  6.5  ...                          5
                               P80                                                 96.0  ...                          5
                               P82                                                 17.5  ...                          5
                               P9                                                   3.0  ...                          1

[3141 rows x 13 columns]

#One Hot Encoding

Pour rappel, le One hot encoding crée des dummy variables qui sont des variables dupliquées représentant un niveau d'une variable catégorielle. La présence d'un niveau est représenté par un 1 et sont absence est représenté par un 0. Si la variable catégorielle est ordinale (i.e. les catégories de la variable ont un ordre) alors on peut traduire celle-ci en une variable numérique en utilisant la fonction `OrdinalEncoder`. Dans notre cas, les variables catégorielles n'ont pas d'ordinalité et donc, on utilise `get_dummies` pour créer les dymmy variables

In [ ]:
cat_vars = ['most_frequent_day', 'most_frequent_colour',
            'most_frequent_location', 'most_frequent_photo_type',
            'most_frequent_price_type', 'most_frequent_page_number']
df2[cat_vars] =df2[cat_vars].astype(str)

df3 = pd.get_dummies(df2)

df3.head(5)

median_no_of_clicks_per_session  ...  most_frequent_page_number_5
country page 1 (main category) page 2 (clothing model)                                   ...                             
1       3                      C30                                                  2.0  ...                            0
        4                      P17                                                  2.0  ...                            0
10      1                      A1                                                   2.0  ...                            0
                               A11                                                  2.0  ...                            0
                               A12                                                  3.0  ...                            0

[5 rows x 67 columns]

On peut aussi utiliser la fonction `OneHotEncoder` à la place de `get_dummies` si nos features nominales sont des entiers.

#Standardisation

Comme on l'a vu, nos variables numériques ont des échelles différents. Le "**scaler**" permet de comparer des variables indépendantes avec différentes unités ou différents **ranges**.

Il y a deux types majeurs de scaling:

1. **Standardisation**: on transforme les variables pour qu'elles aient les propriétés d'une distribution normale standardisée (moyenne = 0, déviation standard = 1). 
En gros z = (x - mu) / sigma
Le désavantage c'est que les données ne sont pas limités par dans un range de valeurs spécifiques.

2. **Normalisation**: les variables sont scalées dans un intervalle fixe [-1 : 1]. X_new= (X - X_min)/(X_max - X_min). Le désavantage étant que s'il y a des outliers dans les données, celles-ci seront mises à l'échelle dans un intervalle très petit.

La normalisation est le choix idéal lorsque l'on sait que la distribution des données ne suit pas une Gaussienne ou pour des algos qui ne font aucune hypothèse sur la distribution des données, comme les K-Nearest Neighbors ou les Neural networks. D'un autre côté, la standardisation peut être utilisée lorsque les données suivent une distribution Gaussienne.

Mais il n'y a pas de règles strictes et idéalement on peut essayer les deux et sélectionner l'option qui donne les meilleurs résultats de validation des clusters. On va standardiser nos champs numériques en utilisant la fonction `StandardScaler`.

In [ ]:
from sklearn.preprocessing import StandardScaler

con_vars = ['median_no_of_clicks_per_session', 'min_no_of_clicks_per_session',
             'max_no_of_clicks_per_session', 'median_price', 'min_price',
             'max_price', 'total_number_of_sessions']

scaler = StandardScaler()
df3[con_vars] = scaler.fit_transform(df3[con_vars])
df3.head(5)

median_no_of_clicks_per_session  ...  most_frequent_page_number_5
country page 1 (main category) page 2 (clothing model)                                   ...                             
1       3                      C30                                            -0.784923  ...                            0
        4                      P17                                            -0.784923  ...                            0
10      1                      A1                                             -0.784923  ...                            0
                               A11                                            -0.784923  ...                            0
                               A12                                            -0.686829  ...                            0

[5 rows x 67 columns]

#Principal Component Analysis (PCA)
Ou Analyse en Composantes Principales (ACP)

L'Analyse en Composantes Principales combine nos variables d'une manière spécifique pour créer de nouvelles variables de manière à nous permettre de "droper" ou d'ignorer les *moins importantes* tout en continuant à garder les parties les plus importantes des variables originales (on se réfère ici aux variables les plus fortement liées à la variabilité des données originales). C'est une méthode utile lorsque l'on a à faire à de nombreuses variables. La matrice des covariances de toutes les variables est calculées et de cette matrice sont déduits les valeurs propres et les vecteurs propres. Ensuite la matrice de covariance est multipliée par ces vecteurs propres pour créer les componsantes principales. Ces composantes principales sont les nouvelles variables (*features*) basées sur nos variables origineles et leur importance pour expliquer la variabilité du jeu de données est donnée par les valeurs propres. On peut garder les composantes principales les mieux classées pour expliquer un maximum de la variance du jeu de données originel en "gardant un minimum de variables".

On peut implémenter l'ACP en utilisant la fonction `pca` du module `sklearn.decomposition`. Nous allons écrire une boucle qui identifie le nombre de composantes principales à même d'expliquer au moins 85% de la variance dans le jeu de données

In [ ]:
from sklearn.decomposition import PCA

#Boucle pour identifier de quoi expliquer au moins 85% de la variance:

for comp in range(3, df3.shape[1]):
  pca = PCA(n_components= comp, random_state=42)
  pca.fit(df3)
  comp_check = pca.explained_variance_ratio_  #oui oui, le nom finit en "_" underscore
  final_comp = comp
  if comp_check.sum() > 0.85:
    break

Final_PCA= PCA(n_components= final_comp, random_state=42)
Final_PCA.fit(df3)
cluster_df=Final_PCA.transform(df3)

num_comps = comp_check.shape[0]

print("En utilisant {} composantes, on explique {}% de la variabilité dans le jeu de données d'origine.".format(final_comp,comp_check.sum()))


En utilisant 15 composantes, on explique 0.8521100128036767% de la variabilité dans le jeu de données d'origine.


On voit que 15 variables sont nécessaire (suffisantes?) pour expliquer 85% de la variance du jeu de données. On peut maintenant utiliser ces variables dans nos modèles non supervisés comme les K-means, DBSCAN, clustering hierarchique etc etc pour segmenter nos produits.